In [165]:
using DataFrames, CSV, JuMP, Gurobi, LinearAlgebra, Random, Printf, StatsBase, CategoricalArrays, Plots, StatsPlots, Distributions

## ____________________________________________________________________

## Import Datasets (Outputted Probabilities and Returns on $100 Bets)

In [166]:
# # add actual data here
# xgb_preds = CSV.read("C:\\Users\\shann\\Documents\\GitHub\\15095-project\\notebooks\\xgb_cv_pred.csv",DataFrame)
# rf_preds = CSV.read("C:\\Users\\shann\\Documents\\GitHub\\15095-project\\notebooks\\rf_pred.csv",DataFrame)
# lgbm_preds = CSV.read("C:\\Users\\shann\\Documents\\GitHub\\15095-project\\notebooks\\lgbm_cv_pred.csv",DataFrame);
# logreg_preds = CSV.read("C:\\Users\\shann\\Documents\\GitHub\\15095-project\\notebooks\\logreg_pred.csv",DataFrame);
# new_xgb_preds = CSV.read("C:\\Users\\shann\\Documents\\GitHub\\15095-project\\notebooks\\new_xgb_pred.csv",DataFrame);
# returns = CSV.read("C:\\Users\\shann\\Documents\\GitHub\\15095-project\\data\\group_stage_betting_odds_final_cleaned.csv",DataFrame);

# # add actual data here
new_xgb_preds = CSV.read("/Users/yenhann/Documents/MIT/15.095 Machine Learning/15095-project/notebooks/new_xgb_pred.csv",DataFrame);
returns = CSV.read("/Users/yenhann/Documents/MIT/15.095 Machine Learning/15095-project/data/group_stage_betting_odds_final_cleaned.csv",DataFrame);

In [167]:
returns_matrix = Matrix(returns[:,3:5]);
# # matrix of probabilities for model's predictions
# xgb_probs = Matrix(xgb_preds[:,3:5]);
# rf_probs = Matrix(rf_preds[:,3:5]);
# lgbm_probs = Matrix(lgbm_preds[:,3:5]);
# logreg_probs = Matrix(logreg_preds[:,3:5]);
new_xgb_probs = Matrix(new_xgb_preds[:,3:5]);

In [168]:
new_xgb_preds

Row,home_team,away_team,home_win,draw,home_loss
,String15,String15,Float64,Float64,Float64
1,Argentina,Mexico,0.691677,0.0415087,0.266814
2,Argentina,Saudi Arabia,0.968919,0.025733,0.00534816
3,Australia,Denmark,0.238239,0.210197,0.551563
4,Belgium,Canada,0.964543,0.0154218,0.0200349
5,Belgium,Morocco,0.86645,0.0244761,0.109074
6,Brazil,Serbia,0.760576,0.0116859,0.227739
7,Brazil,Switzerland,0.907287,0.00921022,0.0835025
8,Cameroon,Brazil,0.00561122,0.175407,0.818982
9,Cameroon,Serbia,0.184903,0.153135,0.661962


In [169]:
returns

Row,home_team,away_team,home_win_earnings_per_dollar,draw_earnings_per_dollar,away_win_earnings_per_dollar,implied_probability_of_home_win,implied_probability_of_draw,implied_probability_of_away_win
,String15,String15,Float64,Float64,Float64,Float64,Float64,Float64
1,Argentina,Mexico,0.549451,2.95,6.12,0.64539,0.338983,0.163399
2,Argentina,Saudi Arabia,0.120048,8.21,24.52,0.892819,0.121803,0.040783
3,Australia,Denmark,6.38,3.88,0.429185,0.15674,0.257732,0.6997
4,Belgium,Canada,0.900901,2.45,3.51,0.526066,0.408163,0.2849
5,Belgium,Morocco,0.581395,3.21,4.92,0.632353,0.311526,0.203252
6,Brazil,Serbia,0.5,3.28,6.6,0.666667,0.304878,0.151515
7,Brazil,Switzerland,0.469484,3.57,6.23,0.680511,0.280112,0.160514
8,Cameroon,Brazil,7.93,4.32,0.359712,0.126103,0.231481,0.73545
9,Cameroon,Serbia,4.3,2.77,0.70922,0.232558,0.361011,0.585062


In [170]:
# check if each row of new_xgb_preds and returns are the same
for i in 1:size(new_xgb_preds,1)
    if new_xgb_preds[i,1] != returns[i,1] || new_xgb_preds[i,2] != returns[i,2]
        println("not the same")
    end
end


## ____________________________________________________________________

In [171]:
function FIFA2022_GS_Bettings(P, R, B, K)
    
    # define model
    model = Model(Gurobi.Optimizer)

    # define parameters and decision variables
    m = size(P, 1)              # 48 group stage matches
    n = size(P, 2)              # 3 outcomes

    @variable(model, x[i=1:m, j=1:n] >= 0) # x[i,j] is the amount of money bet on outcome j of match i
    @variable(model, z[i=1:m, j=1:n], Bin) # z[i,j] is whether you bet on outcome j of match i

    # define objective: maximising expected total returns across all matches
    @objective(model, Max, sum(P[i,1]*R[i,1]*x[i,1] + 
                               P[i,2]*R[i,2]*x[i,2] + 
                               P[i,3]*R[i,3]*x[i,3] - 
                               sum(x[i,j] for j=1:n) for i=1:m))

    # # constraints
    # 1) You can only bet on one outcome for each match
    @constraint(model, [i=1:m], sum(z[i,j] for j=1:n) <= 1)

    # 2) x[i,j] <= B*z[i,j] for all i and j
    @constraint(model, [i=1:m, j=1:n], x[i,j] <= B/12*z[i,j])

    # 3) Budget constraints
    @constraint(model, sum(x[i,j] for j=1:n, i=1:m) <= B)

    # 4) You must spread your risk across at least K matches
    @constraint(model, sum(z[i,j] for j=1:n, i=1:m) >= K)

    # 5) if z = 1, then x > 0
    @constraint(model, [i=1:m, j=1:n], x[i,j] >= 1*z[i,j])

    # solve model
    optimize!(model)

    # get optimal bet amounts (x) and how you bet (z)
    opt_val = objective_value(model)
    x_opt = value.(x)
    z_opt = value.(z)

    return opt_val, x_opt, z_opt
end

FIFA2022_GS_Bettings (generic function with 1 method)

In [172]:
exp_prof, x_opt, z_opt = FIFA2022_GS_Bettings(new_xgb_probs, returns_matrix, 5000, 5);

Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-19
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (mac64[x86])
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 338 rows, 288 columns and 1008 nonzeros
Model fingerprint: 0xa8ed24e9
Variable types: 144 continuous, 144 integer (144 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+02]
  Objective range  [1e-02, 1e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 5e+03]
Found heuristic solution: objective 200.5775076
Presolve removed 278 rows and 200 columns
Presolve time: 0.00s
Presolved: 60 rows, 88 columns, 254 nonzeros
Found heuristic solution: objective 201.6468878
Variable types: 20 continuous, 68 integer (68 binary)
Found heuristic solution: objective 201.7622975

Root relaxation: objective 8.341491e+03, 17 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     W

In [173]:
# which matches to bet on
betting_matches = findall(x -> x >= 1, z_opt)

12-element Vector{CartesianIndex{2}}:
 CartesianIndex(3, 1)
 CartesianIndex(22, 1)
 CartesianIndex(37, 1)
 CartesianIndex(11, 2)
 CartesianIndex(31, 2)
 CartesianIndex(35, 2)
 CartesianIndex(41, 2)
 CartesianIndex(1, 3)
 CartesianIndex(6, 3)
 CartesianIndex(15, 3)
 CartesianIndex(19, 3)
 CartesianIndex(45, 3)

In [174]:
# make betting matches into list of first index of tuples
betting_matches_list = [x[1] for x in betting_matches]
betting_matches_outcomes = [x[2] for x in betting_matches]

betting_matches_list

12-element Vector{Int64}:
  3
 22
 37
 11
 31
 35
 41
  1
  6
 15
 19
 45

In [175]:
x_opt[6,:]

3-element Vector{Float64}:
   0.0
   0.0
 416.66666666666663

In [176]:
returns[6,:]

Row,home_team,away_team,home_win_earnings_per_dollar,draw_earnings_per_dollar,away_win_earnings_per_dollar,implied_probability_of_home_win,implied_probability_of_draw,implied_probability_of_away_win
,String15,String15,Float64,Float64,Float64,Float64,Float64,Float64
6,Brazil,Serbia,0.5,3.28,6.6,0.666667,0.304878,0.151515


In [177]:
outcomes = new_xgb_preds[betting_matches_list,:]

Row,home_team,away_team,home_win,draw,home_loss
,String15,String15,Float64,Float64,Float64
1,Australia,Denmark,0.238239,0.210197,0.551563
2,Iran,USA,0.515694,0.270426,0.21388
3,Saudi Arabia,Mexico,0.557936,0.14692,0.295144
4,Costa Rica,Germany,0.0131862,0.444052,0.542761
5,Poland,Argentina,0.0124997,0.398129,0.589371
6,Qatar,Ecuador,0.326202,0.570737,0.103061
7,Spain,Germany,0.227806,0.54764,0.224554
8,Argentina,Mexico,0.691677,0.0415087,0.266814
9,Brazil,Serbia,0.760576,0.0116859,0.227739


In [178]:
outcomes[:,"bet_amount"] = [x_opt[i] for i in betting_matches];

In [179]:
outcomes[:,"actual_outcome"] = [1,3,3,3,3,3,2,1,1,3,1,2];

In [180]:
outcomes[:,"bet_outcome"] = [x[2] for x in betting_matches];

In [181]:
# is actual_outcome = bet_outcome?
outcomes[:,"correct"] = outcomes[:,"actual_outcome"] .== outcomes[:,"bet_outcome"];

In [182]:
# bet result: if correct == false, then bet_result = -bet_amount, if correct == true, then bet_result = bet_amount*
bet_result = exp_prof - sum(outcomes[:,"bet_amount"].* (outcomes[:,"correct"].==false))

4591.491360000001

In [183]:
betting_matches

12-element Vector{CartesianIndex{2}}:
 CartesianIndex(3, 1)
 CartesianIndex(22, 1)
 CartesianIndex(37, 1)
 CartesianIndex(11, 2)
 CartesianIndex(31, 2)
 CartesianIndex(35, 2)
 CartesianIndex(41, 2)
 CartesianIndex(1, 3)
 CartesianIndex(6, 3)
 CartesianIndex(15, 3)
 CartesianIndex(19, 3)
 CartesianIndex(45, 3)

## money actually earned

In [184]:
outcomes

# add betting_matches_list as a column called :betting_indices so we can map to the correct row and columns in 'returns' dataframe
outcomes[:,"betting_matches_indices"] = betting_matches_list;
outcomes[:,"betting_matches_outcomes"] = betting_matches_outcomes;
outcomes

Row,home_team,away_team,home_win,draw,home_loss,bet_amount,actual_outcome,bet_outcome,correct,betting_matches_indices,betting_matches_outcomes
,String15,String15,Float64,Float64,Float64,Float64,Int64,Int64,Bool,Int64,Int64
1,Australia,Denmark,0.238239,0.210197,0.551563,416.667,1,1,true,3,1
2,Iran,USA,0.515694,0.270426,0.21388,416.667,3,1,false,22,1
3,Saudi Arabia,Mexico,0.557936,0.14692,0.295144,416.667,3,1,false,37,1
4,Costa Rica,Germany,0.0131862,0.444052,0.542761,416.667,3,2,false,11,2
5,Poland,Argentina,0.0124997,0.398129,0.589371,416.667,3,2,false,31,2
6,Qatar,Ecuador,0.326202,0.570737,0.103061,416.667,3,2,false,35,2
7,Spain,Germany,0.227806,0.54764,0.224554,416.667,2,2,true,41,2
8,Argentina,Mexico,0.691677,0.0415087,0.266814,416.667,1,3,false,1,3
9,Brazil,Serbia,0.760576,0.0116859,0.227739,416.667,1,3,false,6,3


In [191]:
# # EXPECTED PROFIT
# create a new column called "if_bets_correct_profit" that calculates the profit if the bets are correct
# initialise new if bets correct profit list
if_bets_correct_profit = zeros(size(outcomes,1))

# for each row in outcomes, calculate the profit if the bets are correct
for i in 1:size(outcomes,1)
    if_bets_correct_profit[i] = outcomes[i,"bet_amount"] * returns[outcomes[i,"betting_matches_indices"],outcomes[i,"betting_matches_outcomes"]+2] - outcomes[i,"bet_amount"]
end

# add if_bets_correct_profit to outcomes dataframe
outcomes[:,"if_bets_correct_profit"] = if_bets_correct_profit;
outcomes


Row,home_team,away_team,home_win,draw,home_loss,bet_amount,actual_outcome,bet_outcome,correct,betting_matches_indices,betting_matches_outcomes,if_bets_correct_profit
,String15,String15,Float64,Float64,Float64,Float64,Int64,Int64,Bool,Int64,Int64,Float64
1,Australia,Denmark,0.238239,0.210197,0.551563,416.667,1,1,true,3,1,2241.67
2,Iran,USA,0.515694,0.270426,0.21388,416.667,3,1,false,22,1,908.333
3,Saudi Arabia,Mexico,0.557936,0.14692,0.295144,416.667,3,1,false,37,1,1391.67
4,Costa Rica,Germany,0.0131862,0.444052,0.542761,416.667,3,2,false,11,2,3820.83
5,Poland,Argentina,0.0124997,0.398129,0.589371,416.667,3,2,false,31,2,1070.83
6,Qatar,Ecuador,0.326202,0.570737,0.103061,416.667,3,2,false,35,2,666.667
7,Spain,Germany,0.227806,0.54764,0.224554,416.667,2,2,true,41,2,591.667
8,Argentina,Mexico,0.691677,0.0415087,0.266814,416.667,1,3,false,1,3,2133.33
9,Brazil,Serbia,0.760576,0.0116859,0.227739,416.667,1,3,false,6,3,2333.33


In [192]:
# # ACTUAL PROFIT
# for each row i in outcomes:
# if correct == true, then actual_profit = if_bets_correct_profit, 
# if correct == false, then actual_profit = -bet_amount

# initialise new actual profit list
actual_profit = zeros(size(outcomes,1))

# for each row in outcomes, calculate the actual profit and store in actual_profit list
for i in 1:size(outcomes,1)
    if outcomes[i,"correct"] == true
        actual_profit[i] = outcomes[i,"if_bets_correct_profit"]
    else
        actual_profit[i] = -outcomes[i,"bet_amount"]
    end
end

# add actual_profit to outcomes dataframe
outcomes[:,"actual_profit"] = actual_profit;
outcomes

Row,home_team,away_team,home_win,draw,home_loss,bet_amount,actual_outcome,bet_outcome,correct,betting_matches_indices,betting_matches_outcomes,if_bets_correct_profit,actual_profit
,String15,String15,Float64,Float64,Float64,Float64,Int64,Int64,Bool,Int64,Int64,Float64,Float64
1,Australia,Denmark,0.238239,0.210197,0.551563,416.667,1,1,true,3,1,2241.67,2241.67
2,Iran,USA,0.515694,0.270426,0.21388,416.667,3,1,false,22,1,908.333,-416.667
3,Saudi Arabia,Mexico,0.557936,0.14692,0.295144,416.667,3,1,false,37,1,1391.67,-416.667
4,Costa Rica,Germany,0.0131862,0.444052,0.542761,416.667,3,2,false,11,2,3820.83,-416.667
5,Poland,Argentina,0.0124997,0.398129,0.589371,416.667,3,2,false,31,2,1070.83,-416.667
6,Qatar,Ecuador,0.326202,0.570737,0.103061,416.667,3,2,false,35,2,666.667,-416.667
7,Spain,Germany,0.227806,0.54764,0.224554,416.667,2,2,true,41,2,591.667,591.667
8,Argentina,Mexico,0.691677,0.0415087,0.266814,416.667,1,3,false,1,3,2133.33,-416.667
9,Brazil,Serbia,0.760576,0.0116859,0.227739,416.667,1,3,false,6,3,2333.33,-416.667


In [193]:
# sum outcomes[:,"actual_earnings"]
sum(outcomes[:,"actual_profit"])

-466.6666666666655